# Revised case normalization for Hirslanden Aarau 2018

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys

import pandas as pd
import numpy as np

from loguru import logger

sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize, remove_leading_zeros
from src.service import bfs_cases_db_service as bfs_db
from src.service.bfs_cases_db_service import session, get_sociodemographics_for_hospital_year, get_earliest_revisions_for_aimedic_ids, get_codes, apply_revisions
from src.revised_case_normalization.py.format_for_grouper import format_for_grouper

ImportError: cannot import name 'get_primary_procedures_codes' from 'src.service.bfs_cases_db_service' (/home/jovyan/work/src/service/bfs_cases_db_service.py)

In [ ]:
file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
        'Hirslanden Aarau',
        '2018',
        ['Aarau 2018'])

print(file_info)

In [ ]:
df_revised_case_d2d = normalize(file_info, 0)


In [ ]:
df_revised_case_d2d.head()

# Match revised DtoD data with the BfS data from the database

In [ ]:
cases_in_db = get_sociodemographics_for_hospital_year(file_info.hospital_name_db, int(file_info.year))
cases_in_db[NORM_CASE_ID_COL] = cases_in_db[CASE_ID_COL].apply(remove_leading_zeros)
print(cases_in_db.head())

In [ ]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)

cols_to_join.remove(PATIENT_ID_COL)

# Merge cases in db with the revised cases

joined = pd.merge(df_revised_case_d2d, cases_in_db,
                  how='left',
                  on=cols_to_join,
                  suffixes=('', '_db'))

joined

In [ ]:
# Print out how many rows could not be matched
unmatched = joined[joined[AIMEDIC_ID_COL].isna()]
num_unmatched = unmatched.shape[0]
logger.info(f'{num_unmatched} rows could not be matched, given {sorted(cols_to_join)}')

In [8]:
original_revision_ids = get_earliest_revisions_for_aimedic_ids(joined[AIMEDIC_ID_COL].values.tolist())
original_cases = get_codes(original_revision_ids)
original_cases

,aimedic_id,revision_id,old_pd,secondary_diagnoses,primary_procedure,secondary_procedures
0,115875,115875,I7024,"[I7020, Z9588, I1090, N184, J4483, ...]",395014,"[395011, 397510, 004C12, 005599, 0043, ...]"
1,116472,116472,I5014,"[I5001, I2513, I2522, N184, I1320, ...]",99B712,"[8945, 860A06, 948X40]"
2,116851,116851,I5001,"[J91, I5013, I7020, L304, N185, ...]",887211,"[887963, 887964, 887975, 3491]"
3,117722,117722,I7024,"[Z9588, N184, Z922]",395011,"[397511, 395021, 397510, 0042, 004B18, ...]"
4,117999,117999,N390,"[B965, R400, G3520, F446, Z936]",992217,[]
5,118215,118215,I495,"[I484, I480, I1090, I2519, I340, ...]",893909,[]
6,119991,119991,N1781,"[Y579, I1100, I480, I5001, G2010, ...]",992909,[]
7,120078,120078,N390,"[B962, A4158, R650, E1190, E788, ...]",992217,[]
8,120279,120279,I7024,"[Z9588, Z921, Z922]",395014,"[004C12, 395011, 0043, 004B1A, 004B46, ...]"
9,121288,121288,I208,"[I2513, D684, U6912, E788]",361226,"[361121, 361122, 361C14, 3963, 3964, ...]"


In [9]:
# Format joined dataset to the SwissDRG Batchgrouper 2017 Format 

grouper_input_data_string = format_for_grouper(joined)

grouper_input_data_string

grouper_input_data_string[17]

2022-10-31 09:24:33.263 | INFO     | src.revised_case_normalization.py.format_for_grouper:format_for_grouper:91 - Formatted 17 cases


KeyError: 17

In [ ]:
import subprocess


example_batch_line = """0044007489;57;0;0|0;W;20190101;01;20190101;00;1;0;C20|Z432|N40|I440|I493;465110::20190101|4823::20190101|009A13::20190101;"""

# add date to procedures
# change ; with | at one spot
test_df_line = """17722;93;;;M;20180315;01;20180317;00;2;0;I7024|Z9588|N184|Z922|I743;395011::|397511::|395021::|397510::|0042::|004B18::|004B1A::|884911::|005599::|0046::|393012::;"""

grouper_result = subprocess.check_output(["java",
                                "-cp",
                                "/home/jovyan/work/resources/aimedic-grouper-assembly-0.0.0-SNAPSHOT.jar",
                                "ch.aimedic.grouper.BatchGroupeOne",
                                grouper_input_data_string[6]
                                #test_df_line,
                                ]).decode("utf-8")


grouperResults = subprocess.check_output(["java",
                                "-cp",
                                "/home/jovyan/work/resources/aimedic-grouper-assembly-0.0.0-SNAPSHOT.jar",
                                "ch.aimedic.grouper.BatchGroupMany",
                                f'{grouper_input_data_string[1]}#{grouper_input_data_string[6]}#{grouper_input_data_string[16]}',
                                "#"
                                #test_df_line,
                                ]).decode("utf-8")

print(grouper_result)

In [ ]:
# TODO:
# - Add medication 
# - Add and compare CHOP Codes for sideness and procedure date